## Generalized Parallel Replica dynamics
#### Author: Noé Blassel

In this hands-on session, you will be manipulating and implementing some mechanisms in the Parallel Replica Algorithm, one of the three accelerated MD algorithms championed by Arthur Voter at Los Alamos in the late 1990's along with Temperature Accelerated Dynamics and Hyperdynamics. More precisely, we will be working with a simple implementation of GenParRep, a variant using a Flemming-Viot process to simultaneously detect convergence to a QSD and prepare nearly <i>iid</i> samples thereof. See [Binder, Lelièvre & Simpson 2015](https://arxiv.org/abs/1105.4636) for a more complete introduction.

In [ ]:
using Pkg
Pkg.add("Parameters")

In [ ]:
using Base.Threads
using Plots, LaTeXStrings, KernelDensity, Statistics

include("../lib/Potentials.jl")

using .Potentials # exports the entropic_switch and grad_entropic_switch methods from Tuesday's notebook

println(nthreads()) # check multithreading availability


### I) Metastability for the reference (sequential) dynamics

We wish to sample trajectories of the overdamped Langevin dynamics
$$\begin{equation}\mathrm{d}X_t = -\nabla V(X_t)\,\mathrm{d} t + \sqrt{\frac2\beta}\mathrm{d}W_t\end{equation}$$
where $W_t$ is a $d$-dimensional Brownian motion, $V:\mathcal D \subset\mathbb{R}^d\to \mathbb{R}$ is the potential energy and $\beta=(k_{\mathrm{B}}T)^{-1}$ is the inverse thermodynamic temperature. These dynamics are known, under mild conditions on the potential or the domain, to be ergodic for the Gibbs measure $$\mu(\mathrm{d} q) = Z_\beta^{-1}\mathrm{e}^{-\beta V(q)}\,\mathrm{d}q.$$. 

We will be using the so-called entropic switch potential on $\mathcal D = \mathbb R^2$ you are familiar with from previous hands-on sessions, which we plot in the cell below.

In [ ]:
xlims = -2.5, 2.5
ylims = -1.75, 2.5
xrange = range(xlims..., 200)
yrange = range(ylims..., 200)

bg_V = contourf(xrange, yrange, entropic_switch, levels=50, cmap=:hsv,aspectratio=1,xlabel=L"x",ylabel=L"y")

The following method to fill out samples a trajectory from the Euler-Maruyama discretization of the dynamics $\text{(1)}$.

In [ ]:

"""
Arguments:
∇V: gradient of potential energy function implementing a valid method for ∇V(q⁰)
dt: timestep
β: inverse temperature
T: simulation time
q⁰: initial point of the trajectory, a d-dimensional vector

Returns:
A d×n Array of the same element type as q⁰, where n = ⌈T/dt⌉ is the number of simulation steps
"""
function em_trajectory(∇V,dt,β,T,q⁰)
    σ = sqrt(2dt/β)

    q = copy(q⁰)
    traj = [copy(q)]

    dims = size(q)

    for i=1:ceil(Int,T/dt)
        # Fill in the Euler-Maruyama update step
        push!(traj,copy(q)) # store a copy of the current state
    end

    return reduce(hcat,traj) # concatenate trajectory points and return
end

<details>
    <summary>
        <b>Click for answer!</b>
    </summary>
    
```julia
    q .+= -dt*∇V(q)+σ*randn(dims)
```

The `.+=` syntax stands for in-place incrementation. The difference with `+=` is that `+=` allocates new memory and binds it to `q`. Although not relevant here, this difference is good to keep in mind when aiming for efficient Julian code.
</details>

Try running the dynamics $T=1,10,500,2000$ starting from an energy minimum. Play with the temperature and observe the results.

In [ ]:
β = 4.0
dt = 5e-3

T = 10 #simulation time

q⁰ = [-1.04805,-0.0420936]

traj = em_trajectory(grad_entropic_switch,dt,β,T,q⁰) # sample a length T trajectory

plot!(deepcopy(bg_V),traj[1,:],traj[2,:],color=:black,alpha=0.2,label=L"\{X_t,\,0\leq t\leq %$T\}") # we use deepcopy to not draw over the original background potential

You should observe that, at sufficiently low temperature, the dynamics remains stuck for long stretches of time inside potential wells of $V$, with rare transition events in which the dynamics moves from one well to another. This is a manifestation of **metastability**, which we can loosely understand as a large timescale gap between the local oscillations of the dynamics inside the well and the exit time to another well.

For the sake of computational efficiency, it may be more adequate to describe the system by passing to a **coarse-grained** description of the dynamics, which we will understand as a partition 
$$ \mathcal D = \bigcup_{k=1}^K \overline{\Omega}_k,\quad \Omega_i \cap \Omega_j = \emptyset,\,i\neq j$$
of the configurational space by means of non-overlapping open subsets, with the $\Omega_k$.

Then the data of:
- The typical probability $P_{ij}$ to transition $\Omega_j$ given the dynamics has entered $\Omega_i$
- The typical residency time $\tau_{ij}$ in $\Omega_i$ given that the next visited state is to $\Omega_j$

can be considered a reasonable coarse-grained description of the kinetic properties of the system.
In addition, we may also be interested in sampling typical **transition states** on $\partial \Omega_i \cap \partial \Omega_j$.

In general, it is also possible to consider generalizations when the $\Omega_j$ do not cover the configurational space, or the case when the states overlap, but we will not discuss these cases here.

### II) Partition into coarse configurational states

In our case, we propose decomposing $\mathcal D = \overline{\Omega}_1 \cup \overline{\Omega}_2 \cup \overline{\Omega}_3$, where each $\Omega_i$ is defined as the intersection of two open half-spaces defined by the three hyperplanes orthogonal to the unstable manifolds at each of the three saddle points of $V$.

More precisely, denote the three minima $x_1$, $x_2$ and $x_3$ corresponding respectively to the left, top and right wells, also $z_{12}$, $z_{23}$ and $z_{13}$ the saddle points, with $z_{ij}$ the saddle point connecting the two wells containing $x_i$ and $x_j$ for $i<j$. Define $H_{ij}$ to be the hyperplane $z_{ij}+E^{+}_{ij}$, where $E^{+}_{ij}$ is the eigenspace of $\nabla^2 V(z_{ij})$ associated with its positive eigenvalue.

Up to some orientation convention on the eigenvectors of the $\nabla^2 V(z_{ij})$, we may define
$$\Omega_1 = H^{-}_{12}\cap H^{+}_{13}\quad,\Omega_2 = H^{+}_{12}\cap H^{+}_{23},\quad \Omega_3 = H^{-}_{13}\cap H^{-}_{23},$$
where $H^{\pm}$ denotes the positive (resp. negative) open half-space for some affine hyperplane $H$.

The set $\Omega_i$ can be understood as a first-order approximation to the bassin of attraction of $x_i$ for the steepest descent dynamics $\dot q = -\nabla V(q)$.

The following picture should clarify the state definition, and the orientation convention we will use.

In [ ]:
minima = [-1.0480549928242202 0.0 1.0480549928242202; -0.042093666306677734 1.5370820044494633 -0.042093666306677734] # 2x3 matrix of local minima coordinates (x_1 | x_2 | x_3)
saddles = [-0.6172723078764598 0.6172723078764598 0.0; 1.1027345175080963 1.1027345175080963 -0.19999999999972246] # 2x3 matrix of saddle point coordinates (z_{12} | z_{23} | z_{13} )
neg_eigvecs = [0.6080988038706289 -0.6080988038706289 -1.0; 0.793861351075306 0.793861351075306 0.0] # 2x3 matrix of eigenvectors u_{ij} associated with negative eigenvalues of H_{ij}=∇²V(z_{ij}) layed out as(u_{12} | u_{23} | u_{13})

scatter!(deepcopy(bg_V),minima[1,:],minima[2,:],label="minima")
scatter!(saddles[1,:],saddles[2,:],label="saddles")
quiver!(saddles[1,:],saddles[2,:],quiver = (0.3neg_eigvecs[1,:],0.3neg_eigvecs[2,:]),color=:black,label="vecs")

Implement a function $S$ which takes a configuration $q$ and outputs $i$ such that $q \in \overline{\Omega}_i$.

In [ ]:
"""
Arguments:
q: a configuration as a two-dimensional vector
Returns:
A integer from {1,2,3} representing the metastable state
"""
function get_state(q)
    # Fill me in
end

<details>

<summary>
<b>Click for answer!</b>
</summary>

```julia
    l1,l2,l3 =[(q-saddles[:,i])'neg_eigvecs[:,i] for i=1:3] # take the scalar product with negative eigenvectors of the Hessian at each saddle point

    # definition of states as intersections of half-spaces
    (l1 <= 0) && (l3 >= 0) && return 1
    (l1 >= 0) && (l2 >= 0) && return 2
    (l3 <= 0) && (l2 <= 0) && return 3
```

Here, we use the `&&` operator, which stands for conditional evaluation, meaning that the expression on the right is evaluated only if the expression on the left evaluates to `true`, similar to Bash
</details>

The dynamics $S_t=S(X_t) \in \{1,\dots,K\}$ is called the **state-to-state dynamics**. Below, we sample a trajectory of the state-to-state dynamics, for a slightly higher temperature.

In [ ]:
β = 2.5 # We slightly heat the system up to observe some transitions
T = 2000

traj = em_trajectory(grad_entropic_switch,dt,β,T,q⁰)
sts = [get_state(traj[:,i]) for i=1:last(size(traj))] # compute the state to state dynamics

plot(0:dt:T,sts,xlabel=L"t",ylabel=L"S_t",label="",ylims=(0,4),linewidth=2)

Note a realization of $S_t$ is fully specified (assuming for simplicity that transition times are isolated in $\mathbb R_+^*$), by a random sequence $(S_n,\tau_n)_{n\geq 1}\in(\{1,\dots,K\}\times{\mathbb R_+^*})^{\mathbb N}$ of states and occupation times. Note that, if $(S_n)_{n\geq 1}$ were a Markov chain and the $(\tau_n)_{n\geq 1}$ were independent exponential variables with $ \tau_k \sim \tau_j$ given $\{S_k=S_j\}$, $S_t$ would be a continuous-time Markov process.

Such models of the coarse-grained dynamics of the system are called **kinetic Monte-Carlo (KMC)** models in the computational chemistry litterature, and in practice Parallel Replica methods, or variants such as [ParSplice](https://pubs.acs.org/doi/full/10.1021/acs.jctc.5b00916), are used to parametrize such models, giving access to record-breaking simulation times. This is however just an approximation, since the true state-to-state dynamics is generally non-Markovian (try to imagine a situation where this is clear!)

### III) Description of the Generalized Parallel Replica algorithm (GenParRep)

Recall that, given a subset of configurations $\Omega \subset \mathcal D$, with the first exit time from $\Omega$
$$ \tau = \inf \{ t\geq 0 : X_t\not\in\Omega\},$$
the quasi-stationnary distribution (QSD) is defined as the following weak limit:
$$\nu = \underset{t\to\infty}{\lim}\, \mathcal{L}\left(X_t\,\middle|\,\tau>t\right).$$

If the set $\Omega$ is metastable, the QSD is a natural notion of a **local equilibrium** within $\Omega$. For the overdamped Langevin dynamics, under mild conditions on $V$ and $\Omega$, it can be shown that the QSD is uniquely defined. 

The key property of the QSD is that, under initial distribution $\nu$, the first exit time $\tau$ is an **exponential** random variable **independent from the exit point** $X_\tau$!

Using this property, we may use the following algorithm (note in the following $\Omega$ varies through the simulation).

<details>

<summary><b style="color:blue"> Parallel Replica Algorithm:</b> click to unfold!</summary>

1. <b>Initialization:</b> Run the dynamics sequentially until the time until the reference walker $X_t$ enters a state $\Omega$, say $\tau_{\rm ini}$. Increment the simulation clock by said $\tau_{\rm ini}$.
2. <b>Decorrelation:</b> Continue to run the reference dynamics until one of the following happens:
    - <b>Succesful decorrelation:</b>The distribution of the reference walker has converged the QSD (up to some fixed tolerance threshold), say after a time $\tau_{\rm corr}$ inside $\Omega$. Increment the simulation clock by $\tau_{\rm corr}$ and proceed to step 3.
    - <b> Unsuccesful decorrelation:</b>The reference walker has exited before $\tau_{\rm corr}$, thus $\tau < \tau_{\rm corr}$. Increment the simulation clock by $\tau$ and proceed to step 1.
3. <b>Dephasing:</b> Prepare $N$ <i>iid</i> samples from $\nu$, say $(X^1_0,\dots,X^N_0)$. Set the parallel clock to 0.
4. <b>Parallel exit:</b> Evolve the $N$ replicas independently (which can be parallelized), until the fastest one (say $X^1$) exits the domain at time $\tau_{\rm par}=\underset{k=1,\dots,N}{\min}\tau^k$. 
5. <b>Transition sampling:</b> Since the $\tau^k$ are <i>iid</i> exponentials, $N\tau_{\rm par}$ is an unbiased estimation of $\mathbb{E}^{\nu}[\tau]$, and $X^1_{\tau_{\rm par}}$ is an unbiased sample of the transition state (by the key property of the QSD). Increment the simulation time by $N\tau_{\rm par}$, set the state of the reference walker to $X^1_{\tau_{\rm par}}$, and proceed to step 1.

</details>

Two questions remain:
- How to detect decorrelation to the QSD?
- How to prepare <i>iid</i> samples according to the QSD?

While original formulations of Parallel Replica used a deterministic time $\tau_corr$ using heuristics from harmonic approximations of the potential well, the Generalized Parallel Replica approach proposes combining steps 2 and 3 in the algorithm above, without a priori knowledge of the typical decorrelation time, using a **Flemming-Viot** process and a convergence criterion for Markov chains, the **Gelman-Rubin diagnostic**.

<details>

<summary><b style="color:blue"> Flemming-Viot process:</b> click to unfold!</summary>

The [Flemming-Viot process](https://people.bordeaux.inria.fr/pierre.delmoral/fleming-viot-1979.pdf) is a branching and interacting particle process which we describe informally by the following procedure. Fix a number of replicas $N>1$, and consider the following process:x
1. Spawn $N$ realizations $\left(X^{(k)}\right)_{k=1,\dots,m}$ of the dynamics from an initial point inside $\Omega$, driven by independent Brownian motions, with associated exit times $\tau_\Omega^{k}$.
2. At the first time a replica exits $\Omega$, $\tau_{\Omega}^I$, with $I = \underset{k=1,\dots,N}{\mathrm{argmin}}\,\tau_\Omega^k$, pick $J$ uniformly at random in $\{1,\dots,N\}\setminus \{I\}$
3. Branch the$I$-th replica from $X^{(J)}_{\tau_\Omega^{(I)}}$, and evolve the replicas under independent Brownian motions, until the second time a replica exits~$\Omega$, and so on.

The procedure is continued, yielding a process ${\bf X} = (X^{(1)},\dots,X^{(N)})$ on $\Omega^N$, called the Flemming-Viot process. It can be shown under many conditions that, almost surely, the empirical distribution 
$$\frac1N\sum_{n=1}^N \delta_{X^{(n)}_t} \overset{\text{weakly}}{\underset{t\to\infty}{\longrightarrow}} \nu.$$
These type of results are known as propagation of chaos properties, see for instance [arXiv:2207.02030](https://arxiv.org/abs/2207.02030) for references and proofs in a case close to our situation.

<b>Caveat:</b> Since the particles are interacting, the $X^{(n)}_t$ are not independent for large $t$. However, it is reasonable, if exits from $\Omega$ are rare, and for large enough $N$, to consider that they are sufficiently weakly correlated so that for large $t$, $(X^{(1)},\dots,X^{(N)})_t$ can be effectively be considered to be a sample from $\nu^{\otimes N}$.
</details>

<details>

<summary><b style="color:blue"> Gelman-Rubin diagnostic:</b> click to unfold!</summary>

The Gelman-Rubin diagnostic is a tool that gives a necessary condition for the decorrelation of independent Markov processes starting from a common initial point.

Let $(X^{(1)},\dots,X^{(N)})$ be <i>iid</i> diffusions in $\mathcal D$.
Fix observables $\phi_k : \mathcal D \to \mathbb R,\,1\leq k \leq m$, and define

$$ \overline{\phi}_{k,T}^{(n)} = \frac1{T}\int_0^T \phi_k(X^{(n)}_s)\,\mathrm{d}s,\quad \overline{\phi}_{k,T} = \frac{1}{N}\sum_{n=1}^N \overline{\phi}_{k,T}^{(n)}.$$
Thus $\phi_{k,T}^{(n)}$ is the ergodic average at time $T$ of $\phi_k$ under the dynamics of $X^{(n)}$, and $\overline{\phi}_{k,T}$ is the average over all replicas.
The Gelman-Rubin ratio is defined by:

$$ R_k(T) = \frac{\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}_{k,T}\right)^2\,\mathrm{d}t}{\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}^{(n)}_{k,T}\right)^2\,\mathrm{d}t}=1+\frac{T\sum_{n=1}^N(\overline{\phi}_{k,T}^{(n)}-\overline{\phi}_{k,T})^2}{\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}^{(n)}_{k,T}\right)^2\,\mathrm{d}t}\geq 1$$
If ergodicity holds pathwise, this ratio converges to $1$ as $T\to\infty$. The Gelman-Rubin diagnostic stipulates, for a given tolerance $\alpha > 0$, that the $X^{(i)}$ have decorrelated when 
$$\underset{k=1,\dots,m}{\max} R_k(T) < 1+\alpha.$$

In practice, one chooses $\alpha$ in the range $[0.01,0.2]$.

<b> Caveat 1:</b>This criterion is sensitive to the choice of the $\phi_k$. In practice one has to be careful to choose observables which see physically relevant slow variables of the underlying process. As we have no <i>a priori</i> access to these slow variables, the diagnostic only gives a necessary, but not sufficient condition for convergence to equilibrium.

<b> Caveat 2:</b> Strictly speaking, the Gelman-Rubin diagnostic only applies to independently-driven processes. We assume that exits from $\Omega$ are sufficiently rare so that the Gelman-Rubin diagnostic can be applied to the Flemming-Viot process.

</details>

Using these two ingredients, one can construct a combined decorrelation/dephasing step for the Generalized Parallel Replica algorithm.

<details>
<summary><b style="color:blue">Combined decorrelation/dephasing step for GenParRep:</b> click to unfold!</summary>

<b> Decorrelation/Dephasing:</b> Spawn a Flemming-Viot process $(X^{(1)},\dots,X^{(N)})$ with $N$ replicas from the reference walker's configuration, such that $X^{(1)}$ follows exactly the dynamics of the reference walker. Evolve the Flemming-Viot process (this can be done in parallel), until one of the following happens:
- <b>Succesful decorrelation:</b>The Flemming-Viot process has reached stationarity, as assessed by the Gelman-Rubin diagnostic, after a time $\tau_{\rm corr}$, such that the replica $X^{(1)}$ has never been killed. Then the reference walker is, by construction approximately distributed according to $\nu$, and one can proceed to step 3 using $(X^{(1)},\dots,X^{(N)})_{\tau_{\rm corr}}$ as approximately <i>iid</i> samples from the QSD, after incrementing the simulation clock by $\tau_{\rm corr}$.
- <b>Unsuccesful decorrelation:</b>The reference walker $X^{(1)}$ has exited $\Omega$ at some time $\tau$ before decorrelation: then, interrupt the Flemming-Viot, increment the simulation clock by $\tau$ and proceed to step 1 starting from $X^{(1)}_\tau$.
</details>

The computational gain of the Parallel Replica method is an expected factor of $N$ gained in the world-clock time spent in the parallel exit step 4 (assuming perfect parallel efficiency). On the other hand, the number CPU cycles expended in the decorrelation/dephasing step is $N$ times greater than one would have needed to compute equivalent portions of the dynamics in a sequential manner. Importantly, the algorithm is only efficient in terms of wall-clock time if $\tau_{\rm corr} \ll \tau$ in $\Omega$: it is thus crucial to define the metastable states in such a way that this property holds.

### IV) Implementation of the Flemming-Viot process

We propose to implement a discrete time version of the Flemming-Viot process, associated with an Euler-Maruyama discretization of the dynamics. It corresponds to the following scheme. Take an initial point,
$$(X^{(1),k},\dots,X^{(N),k}),$$

and sample
$$(\widetilde{X}^{(1),k+1},\dots,\widetilde{X}^{(N),k+1})$$
using a component-wise Euler-Maruyama scheme with independent Brownian increments. Finally: setting
$$ I = \{1\leq n\leq N:\,\widetilde{X}^{(n),k+1}\not\in \Omega \},$$
define the scheme through
$$X^{(n),k+1}=\begin{cases}\widetilde{X}^{(n),k+1},\,\text{if }n\not\in I,\\ \text{sample }X^{(n),k+1}\sim\mathcal U(\{\widetilde{X}^{(n),k+1}\}_{n\not\in I})\,\text{otherwise.}\end{cases}$$

Implement this scheme in the cell below, for $\Omega=\Omega_1$, with initial starting configuration $x_1$.

In [ ]:
"""
Arguments:
∇V: gradient of potential energy function implementing a valid method for ∇V(q⁰)
dt: timestep
β: inverse temperature
T: simulation time
q⁰: initial point of the trajectory, a d-dimensional vector
N: number of Flemming-Viot particles
Tcorr: correlation time (after which samples are collected)
Returns:
A d×n Array of the same element type as q⁰, where n = ⌈T/dt⌉-⌈Tcorr/dt⌉ is the number of simulation steps
"""
function flemming_viot(∇V,dt,β,T,q⁰,N,Tcorr)
    X = [copy(q⁰) for k=1:N]
    n_steps = ceil(Int,T/dt)

    σ = sqrt(2dt/β)
    dims = size(q⁰)

    samps = typeof(q⁰)[]

    ncorr = ceil(Int,Tcorr/dt)

    for i=1:n_steps
        ## Fill me in
        
        (i > ncorr) && append!(samps,X) # start sampling after the correlation time
    end

    return reduce(hcat,samps)
end

<details>

<summary>
<b>Click for answer!</b>
</summary>

```julia
    @threads for k=1:N
        X[k] += -dt*∇V(X[k])+σ*randn(dims)
    end

    killed = @. get_state(X) != 1

    X[killed] .= rand(X[.! killed],sum(killed))
```

The `@threads` macro is a minimalist interface to multithreading, splitting the iterations of the `for` loop between the logical cores available to the Julia instance.
</details>

In the following cell, we plot samples from the Flemming-Viot stationnary distribution to test the implementation above. Try to play around with the temperature parameter in the $\beta\in[1,4]$ range, and observe how the estimated QSD behaves.

In [ ]:
β = 2.0 # play with β
Tcorr = 3
T = 3Tcorr
N = 512

samps = flemming_viot(grad_entropic_switch,dt,β,T,q⁰,N,Tcorr)
plot(histogram2d(samps[1,:],samps[2,:],aspectratio=1,xlabel=L"x",ylabel=L"y",xlims=xlims,ylims=ylims,normalize=true),bg_V,size=(1000,500))

### V) Estimating decorrelation times with Gelman-Rubin diagnostics

We now turn to the implementation of the Gelman-Rubin diagnostic. For now, we will implement it for $N$ independent replicas of the Euler-Maruyama discretization of the reference dynamics. Before this, we need to adress two points:

- Since we have a discrete-time Markov chain, we need a formulation in terms of discrete trajectory points: this is obtained by simply replacing the continuous-time integrals by sums over trajectory points.
- The naive implementation of the Gelman-Rubin diagnostic stores the full numerical trajectories to compute the ratios. However, expanding the numerator and denominator shows that it is sufficient to keep track of the

$$S_{k,T}^{(n)}=\int_0^T \phi_k(X_s^{(n)})\,\mathrm{d} s,\quad Q_{k,T}^{(n)}=\int_0^T \phi_k(X_s^{(n)})^2\,\mathrm{d}s,\quad$$
which may be updated online. Indeed,

$$R_k(T)=\frac{\sum_{n=1}^N Q_{k,T}^{(n)}-\frac1{NT}\left(\sum_{n=1}^N S_{k,T}^{(n)}\right)^2}{\sum_{n=1}^N \left(Q_{k,T}^{(n)}-\frac1TS_{k,T}^{(n)2}\right)}$$

<details>

<summary><b style="color:blue"> Derivation:</b> click to unfold!</summary>

Start from

$$R_k(T) = \frac{\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}_{k,T}\right)^2\,\mathrm{d}t}{\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}^{(n)}_{k,T}\right)^2\,\mathrm{d}t}.$$

Expand the denominator and group terms to get

$$\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}^{(n)}_{k,T}\right)^2\,\mathrm{d}t = \sum_{n=1}^N \int_0^T \phi_k(X_t^{(n)})^2\,\mathrm{d}t - T\overline{\phi}_{k,T}^{(n)2} = \sum_{n=1}^N \left(Q_{k,T}^{(n)}-\frac1TS_{k,T}^{(n)2}\right).$$

Similar straightforward manipulations lead to

$$\sum_{n=1}^N\int_0^T \left(\phi_k(X^{(n)}_t)-\overline{\phi}_{k,T}\right)^2\,\mathrm{d}t = \sum_{n=1}^N Q_{k,T}^{(n)}-\frac1{NT}\left(\sum_{n=1}^N S_{k,T}^{(n)}\right)^2$$

This allows to compute the diagnostic at each step at a cost $\mathrm{O}_{T}(1),\,\mathrm{O}_{N}(N)$.

</details>

Fill in the method in the cell below

In [ ]:
"""
Arguments:
∇V: gradient of potential energy function implementing a valid method for ∇V(q⁰)
dt: timestep
β: inverse temperature
T: simulation time
q⁰: initial point of the trajectory, a d-dimensional vector
N: number of independent replicas for the Markov chain
ϕs: a vector of K functions, such that ϕs[k] implements a method for ϕs[k](q⁰).

Returns:
A K×n Array of Gelman-Rubin ratios, where n = ⌈T/dt⌉ is the number of simulation steps
"""
function gelman_rubin(∇V,dt,β,T,q⁰,N,ϕs)
    samps = [[copy(q⁰)] for k=1:N]
    nsteps = ceil(Int,T/dt)

    dims = size(q⁰)
    σ = sqrt(2dt/β)

    X = [copy(q⁰) for k=1:N]

    K = length(ϕs)

    S = zeros(K,N)
    Q = zeros(K,N)

    Rhist = Vector{Float64}[] # the syntax T[] evaluates to an empty vector with element type T

    for t=1:nsteps
        @threads for n=1:n
            X[n] = -dt*∇V(X[n]) + σ*randn(dims)
            # Implement the computation of the Gelman-Rubin ratios and push to Rhist
        end
            
    end

    return reduce(hcat,Rhist)

end

<details>

<summary>
<b>Click for answer!</b>
</summary>

```julia
    @threads for n=1:N
        X[n] += -dt*∇V(X[n]) + σ*randn(dims)
        for k=1:K
            ϕ = ϕs[k]
            v = ϕ(X[n])
            S[k,n] += v
            Q[k,n] += v^2
        end

        
    end
    R = [(sum(Q[k,n] for n=1:N)-sum(S[k,n] for n=1:N)^2/(N*t))/(sum(Q[k,n]-S[k,n]^2/t for n=1:N)) for k=1:K]
    push!(Rhist,R)
```

Similar to Python, it is possible to build iterators using generator-like expressions, and comprehensions to build vectors. However, it much more efficient in Julia.
</details>

In [ ]:
ϕs = [q->q[1],q->q[2],entropic_switch]
β = 1.0
N = 1000
T = 50

Rhist = gelman_rubin(grad_entropic_switch,dt,β,T,q⁰,N,ϕs)


In [ ]:
plot(dt:dt:T,Rhist[1,:],label=L"x",yaxis=:log,xlabel=L"T",ylabel=L"R(T)")
plot!(dt:dt:T,Rhist[2,:],label=L"y")
plot!(dt:dt:T,Rhist[3,:],label=L"V")

Observe that neglecting $x$ from the Gelman-Rubin diagnostic would have lead to a spurious convergence detection.

### VII) Putting everything together

You have now implemented all the ingredients to have a working GenParRep algorithm, which can in principle be made compatible with arbitrary Markov chains and metastable state definitions. 

If you are interested, I present below a more complete implementation (but by no means optimally efficient!!) able to accomodate more general situations such as exit events defined by an exponential killing rate, or overlapping states, as well as arbitrary Markov chains.

<details>

<summary> <b style="color:blue"> A peak at a fuller implementation: </b> click to unfold! </summary>
The main object is the `GenParRep` algorithm generic type, which has the following structure:

```julia
    Base.@kwdef mutable struct GenParRepAlgorithm{S,P,M,K,R,X,L}

        ## Mechanisms

        simulator::S # a method to evolve the microscopic dynamics
        dephasing_checker::P # an object to check if replicas have decorrelated/dephased
        macrostate_checker::M # an object to check the macrostate
        replica_killer::K # an object to kill the replicas

        logger::L # an object to extract data during the simulation


        ### Internal parameters

        N::Int #number of replicas

        rng::R = Random.default_rng() # PRNG generator
        reference_walker::X # The reference walker
        replicas::Vector{X} = typeof(reference_walker)[] # A vector to hold the replicas

        n_initialisation_ticks::Int=0 # number of calls to simulator in the initialisation step
        n_dephasing_ticks::Int=0 # number of (parallel) calls to the simulator in the dephasing/decorrelation step
        n_parallel_ticks::Int=0 # number of (parallel) calls to the simulator in the parallel step

        n_transitions::Int=0 # number of observed transitions
        simulation_time::Int=0 # simulation time -- in number of equivalent steps of the sequential Markov chain
        wallclock_time::Int=0 # simulation time -- each parallel step is counted as one (this is naive)
    end
```

each of the "Mechanisms" corresponds to a user-defined type, which must implement the following methods:

```julia
        """
        Parameters:
        - checker: the user-defined decorrelation/dephasing checker.
        - replicas: a vector of replicas from the Flemming-Viot process
        - current_macrostate: the current metastable state of the reference dynamics
        - step_n: the number of elapsed Markov chain steps since the last decorrelation/dephasing phase began

        Returns:
        true/false::Bool declaring whether the Flemming-Viot has converged
        """
        ParRep.check_dephasing!(checker,replicas,current_macrostate,step_n)

        """
        Parameters:
        - checker: the user-defined metastable state checker.
        - walker: a vector, the state of the reference walker or one of its replicas
        - current_macrostate: the current metastable state of the reference dynamics
        - step_n: the number of elapsed Markov chain steps since the current simulation phase began
        Returns:
        the metastable state in which the walker is in, or `nothing`::Nothing if the walker is not in a metastable state
        """
        ParRep.get_macrostate!(checker,walker,current_macrostate,step_n)

        """
        Parameters:
        - simulator: the user-defined simulator.
        - walker: a vector, the state of the reference walker or one of its replicas
        Returns:
        Nothing, modifies the state of the walker in place according to one step of the Markov Chain (RNG is the responsibility of the simulator)
        """
        ParRep.update_microstate!(simulator,walker)

        """
        Parameters:
        - checker: the user-defined replica killer.
        - macrostate_a: the current valid metastable state of the reference dynamics or one of its replicas, or `nothing` if the reference walker is not in a metastable state
        - macrostate_b: the tentative new metastable state of the reference dynamics or one of its replicas
        Returns:
        true/false::Bool declaring whether the replica has escaped macrostate_a. For stochastic state definitions, RNG is responsibility of checker.
        """
        ParRep.check_death(checker,macrostate_a,macrostate_b)

        """
        Parameters:
        - logger: the user-defined logger.
        - event: a Symbol ∈ [:initialization,:dephasing,:parallel,:transition] describing the state of the simulation
        - kwargs... allowing for flexible behaviour. Available kwargs vary according to the event.
        Returns:
        Nothing, modifies the state of the logger in-place to output/process data.
        """
        ParRep.log_state!(logger,event; kwargs...)
```

To sample the dynamics, we define the method

```julia
    ParRep.simulate!(alg::GenParRepAlgorithm, n_transitions)
```
which samples `n_transitions` between metastable states for the dynamics defined by `alg`, and returns two integers, `alg.simulation_time` and `alg.wallclock_time`.

Have a look at `../lib/ParRep.jl` and see if you can recognize the structure of the algorithm!

</details>

For now, we will apply the GenParRep algorithm to a simple problem, the sampling of the exit event from $\Omega_1$, starting from $x_1$. That is, we aim to sample

$$ (\tau_1,X_{\tau_1}),\quad \tau_1 = \inf\{t\geq 0,\, X_t \not\in \Omega_1\}$$

with initial law $\delta_{x_1}$.


We provide the following two functions, 


In [ ]:

"""
Arguments:
∇V: gradient of potential energy function implementing a valid method for ∇V(q⁰)
dt: timestep
β: inverse temperature
q⁰: initial point of the trajectory, a d-dimensional vector

Returns:
A (t,q)::Tuple{Float64,Vector{Float64},Int} containing the exit time, the exit configuration and the number of steps of the Markov chain
"""
function sample_exit_dns(∇V,dt,β,q⁰)
    σ = sqrt(2dt/β)
    q = copy(q⁰)
    dims=size(q)
    step = 0

    while get_state(q) == 1
        step +=1
        q .+= -dt*∇V(q)+σ*randn(dims)
    end

    return (step*dt,q,step)

end

"""
Arguments:
∇V: gradient of potential energy function implementing a valid method for ∇V(q⁰)
dt: timestep
β: inverse temperature
T: simulation time
q⁰: initial point of the trajectory, a d-dimensional vector
N: number of replicas for the Flemming-Viot
gr_α: tolerance parameter for the Gelman-Rubin diagnostic
ϕs: a vector of K functions, such that ϕs[k] implements a method for ϕs[k](q⁰).

Returns:
A tuple (t,q,decorr_steps,parallel_steps)::Tuple{Float64,Vector{Float64},Int,Int} containing the exit time, exit configuration, the number of decorrelation/dephasing steps and the number of parallel exit steps.
"""
function sample_exit_genparrep(∇V,dt,β,q⁰,N,gr_α,ϕs)
    σ = sqrt(2dt/β)
    q = copy(q⁰)
    dims=size(q)

    decorr_step = 0
    parallel_step = 0

    X = [copy(q) for i=1:N]

    K = length(ϕs)

    S = zeros(K,N)
    Q = zeros(K,N)

    decorr = false

    # decorrelation/dephasing step
    while !decorr
        decorr_step += 1
        
        @threads for n=1:N # update replicas
            X[n] += -dt*∇V(X[n])+σ*randn(dims)
        end

        killed = @. get_state(X) != 1 # compute states

        if killed[1] # failed decorrelation: reference walker has exited
            return (decorr_step*dt,X[1],decorr_step,parallel_step)
        end

        X[killed] .= rand(X[.! killed],sum(killed)) # Flemmin-Viot branching

        # update Gelman-Rubin quantities
        @threads for n=1:N
            for k=1:K
                ϕ = ϕs[k]
                v = ϕ(X[n])
                S[k,n] += v
                Q[k,n] += v^2
            end  
        end

        if maximum((sum(Q[k,n] for n=1:N)-sum(S[k,n] for n=1:N)^2/(N*decorr_step))/(sum(Q[k,n]-S[k,n]^2/decorr_step for n=1:N)) for k=1:K) < 1 + gr_α # Gelman-Rubin diagnostic
            decorr = true
        end
    end

    # parallel step

    while true
        parallel_step += 1

        @threads for n=1:N # update replicas
            X[n] += -dt*∇V(X[n])+σ*randn(dims)
        end

        killed = @. get_state(X) != 1 # compute states
        
        if any(killed) # succesful exit
            ifirst = argmax(killed)
            return ((decorr_step+N*parallel_step+ifirst)*dt,X[ifirst],decorr_step,parallel_step)
        end

    end
end

In the following cell, we sample `n_exits` exit events, both using direct numerical simulation (DNS) and the GenParRep algorithm.
Sample exit events for both methods: if it is too slow, you can reduce the number of time steps, turn the temperature up, or play with the Gelman-Rubin tolerance, although this will degrade the quality of the samples.

<b style="color:red"> BEWARE: the following cell takes ~10mins to run on my laptop with 8 logical threads. </b>

In [ ]:
n_exits = 1000

times_gpr = Float64[]
times_dns = Float64[]

exits_gpr = Vector{Float64}[]
exits_dns = Vector{Float64}[]

decorr_steps = Int[]
parallel_steps = Int[]
dns_steps = Int[]

N = 128
α = 0.1 # Gelman-Rubin tolerance threshold
β = 2.5

for i=1:n_exits
    (i%20 == 0) && println("Sampling $i-th exit")
    t,x,ds,ps = sample_exit_genparrep(grad_entropic_switch,dt,β,q⁰,N,α,ϕs)
    push!(times_gpr,t)
    push!(exits_gpr,x)
    push!(decorr_steps,ds)
    push!(parallel_steps,ps)
    t,x,n = sample_exit_dns(grad_entropic_switch,dt,β,q⁰)
    push!(times_dns,t)
    push!(exits_dns,x)
    push!(dns_steps,n)
end

- Make plots of $\mathbb P^{x_1}(\tau_1>t)$ as a function of $t$. Try a semi-log axis and comment, for both for the DNS times and the GenParRep times.

- Estimate the average speedup (in the idealization of zero communication cost between replicas) by estimating
$$ \frac{\mathbb{E}^{x_1}\left[n_{\rm exit}^{\rm DNS}\right]}{\mathbb{E}^{x_1}\left[n_{\rm corr}^{\rm GPR}+n_{\rm par}^{\rm GPR}\right]}$$
where $n_{\rm corr}^{\rm GPR}$ is the number of steps spent in the decorrelation/dephasing phase and $n_{\rm par}^{\rm GPR}$ is the number of steps spent in the parallel phase for the parallel replica algorithm $N$, and $n_{\rm exit}^{\rm DNS}$ is the number of steps to exit for the sequential Markov chain. 

- Estimate $\mathbb P^{x_1}(X_{\tau_1} \in \partial \Omega_i)$ for $i=2,3$

- Estimate the proportion of succesful decorrelations

- <b>Hint A:</b> The expression `mean(X .> a)` evaluates to the empirical probability that `X` is greater than `a`
- <b>Hint B:</b> The expression `plot(f,a,b)` plots the scalar-valued function `f` between `a` and `b`

In [ ]:
# Fill me in

<details>
<summary><b>Click for answer!</b></summary>


```julia
P(hist,t) = mean(hist .> t)
println("Estimated maximal speedup: ", mean(dns_steps)/mean(decorr_steps + parallel_steps))

states_dns = get_state.(exits_dns)
states_gpr = get_state.(exits_gpr)
println("P(Xτ = 2) ≈ ",mean(states_dns .== 2) , " (DNS)")
println("P(Xτ = 2) ≈ ",mean(states_gpr .== 2) ," (GPR)")

println("Estimated proportion of succesful decorrelations: ", 1-mean(parallel_steps .== 0))

plot(t->P(times_dns,t),0,100,label="DNS",xlabel=L"t",ylabel=L"\mathbb{P}(\tau_1 > t)",yaxis=:log)
plot!(t->P(times_gpr,t),label="GPR")
```
</details>

Parametrize the boundary of $\Omega_1$ by a function $\xi:\mathbb{R}\to \partial\Omega_1$, and plot histograms of $\xi(X_{\tau_1})$ for both methods